In [166]:
import ml
reload(ml)
from ml import *
import csv
import timeit
import scipy
import operator
import numpy as np
import pandas as pd
import cPickle as pickle
from scipy import stats
import seaborn as sns
from datetime import datetime
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from pprint import pprint
from tqdm import tqdm, tqdm_notebook
from __future__ import division
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
plt.style.use('classic')
%matplotlib inline

__author__ = 'HK Dambanemuya'
__version__ = 'Python 2'

'''
    Analysis originaly performed in Python 2 (deprecated)
    Seaborn, Statsmodel, and * imports broken in Python 3
'''

## Methods

In [167]:
def cov(arr):
    return np.std(arr) / np.mean(arr) 

def fundingTime(arr):
    return (pd.to_datetime(arr.max()) - pd.to_datetime(arr.min())).days

def interEventTime(arr):
    return [(pd.to_datetime(t) - pd.to_datetime(s)).seconds for s, t in zip(sorted(arr), sorted(arr)[1:])]

def avgInterEventTime(arr):
    return np.mean(interEventTime(arr))

def covInterEventTime(arr):
    varr = interEventTime(arr)
    return np.std(varr) / np.mean(varr) 

def truncate_date(x):
    if x.date:
        try:
            return x.strftime('%Y%m')
        except:
            return None

## Data

In [168]:
investments = pd.read_csv("../Data/full.csv")
investment_status = pd.read_csv("../Data/investment_status.csv")
investment_status["ProjectID"] = investment_status.ProjectID.astype(int)
investment_status["Status"] = investment_status.Status.astype(int)
# Create campaigns dataframe
campaigns = investments[["campaign", "premoneyvaluation", "percentageofequityoffered", "eiscompliant",
                           "seiscompliant", "selfcertification", "slug", "passedquiz"]]
# Create investments dataframe
investments = investments[["campaign", "investorid", "amount", "created", "approvedat", "investmentsought"]]
projectsgeq4 = np.loadtxt('projectsgeq4contributions.out', delimiter=',', dtype=int)
features_5 = ["NumContributors", "CovInterEventTime", "CovContributionAmount", "TimeToFirstContribution (sec)", 'Duration (days)']

project_amounts = pd.read_csv('../Data/investment_amounts.csv', header=None)
k = project_amounts.to_dict()[0].values()
v = project_amounts.to_dict()[1].values()

project_amounts = dict(zip(k,v))

# Initialize Standard Scaler
scaler = StandardScaler()

## Campaigns

In [169]:
# Additional variables
campaigns['eiscompliant'] = campaigns['eiscompliant'].astype('category').cat.codes
campaigns['seiscompliant'] = campaigns['seiscompliant'].astype('category').cat.codes
campaigns['selfcertification'] = campaigns['selfcertification'].astype('category').cat.codes
campaigns['slug'] = campaigns['slug'].astype('category').cat.codes
campaigns.passedquiz[campaigns.passedquiz == "FALSE"] = False
campaigns.passedquiz[campaigns.passedquiz == "TRUE"] = True
campaigns.percentageofequityoffered.fillna(0)
campaigns.sample(5)

,campaign,premoneyvaluation,percentageofequityoffered,eiscompliant,seiscompliant,selfcertification,slug,passedquiz
26802,6754,450000000.000,10.000,1,0,2,1894,NaN
85762,NaN,nan,nan,0,0,-1,-1,NaN
51880,9874,120000000.000,8.000,0,1,-1,670,True
1679,866,45000000.000,10.000,1,0,1,934,True
44686,9008,nan,10.000,1,0,1,166,True


### Campaigns Aggregation

In [170]:
g = {'campaign': ['max'],
     'premoneyvaluation': ['max'],
     'percentageofequityoffered': ['max'],
     'eiscompliant': ['max'],
     'seiscompliant': ['max'],
     'selfcertification': ['min'],
     'slug': ['max'],
     'passedquiz': ['max']}

In [171]:
campaigns = pd.DataFrame(campaigns.groupby('campaign').agg(g).as_matrix())
campaigns = campaigns.rename(index=str, columns={0: "PassedQuiz",
                                                 1: "EquityPercent",
                                                 2: "PreMoneyValuation",
                                                 3: "SelfCertification",
                                                 4: "ProjectID",
                                                 5: "EisComplinat",
                                                 6: "SeisCompliant",
                                                 7: "Slug"})
campaigns = campaigns[campaigns.PassedQuiz != 'Consumer Products,E-Commerce & Markets']
campaigns["ProjectID"] = campaigns.ProjectID.astype(int)
campaigns = campaigns.merge(investment_status, on="ProjectID", how="left")
campaigns.head()

,PassedQuiz,EquityPercent,PreMoneyValuation,SelfCertification,ProjectID,EisComplinat,SeisCompliant,Slug,Status
0,True,30.000,23000000.000,-1,15,0,1,2437,1.000
1,True,8.570,160000000.000,-1,16,0,1,1940,0.000
2,True,18.000,68000000.000,-1,35,0,1,1124,0.000
3,True,10.750,58000000.000,-1,37,0,1,2682,0.000
4,True,6.000,39000000.000,-1,38,0,1,358,0.000


## Investments

In [172]:
# Remove bad data
investments = investments[investments.amount != '9/11/2013 23:26']
# Convert created date to datetime format
investments["created"] = pd.to_datetime(investments.created)
# Variable /YYYY/MM to allow monthly time unit analysis
investments["time_trunc"] = investments.created.apply(truncate_date)
# Convert variable to float
investments['amount'] = investments.amount.astype('float')
# Rename columns to be readable and understandable
investments = investments.rename(index=str, columns={'created': "investment_timestamp", 
                                                     'approvedat': "approved_timestamp",
                                                     'investmentsought': "amount_requested"})
investments = investments.dropna()
# Show sample data
investments.sample(5)

,campaign,investorid,amount,investment_timestamp,approved_timestamp,amount_requested,time_trunc
47507,9161,33936.000,30.000,2015-05-19 10:00:00,4/9/2015 16:42,60000.000,201505
2785,1299,8082.000,10.000,2013-02-11 12:47:00,12/21/2012 10:13,75000.000,201302
75164,13303,107636.000,105.480,2015-11-19 19:02:00,11/11/2015 13:22,696402.000,201511
57402,10451,59365.000,1000.000,2015-05-07 13:21:00,4/20/2015 14:52,55000.000,201505
29501,7228,57660.000,750.000,2014-10-15 14:30:00,9/24/2014 18:12,400000.000,201410


In [173]:
print len(investments)

77419


In [174]:
print(len(investments.campaign.unique()))

740


### Filter Investments

In [175]:
investments.head(10).sort_values(by="investment_timestamp")

,campaign,investorid,amount,investment_timestamp,approved_timestamp,amount_requested,time_trunc
3,15,12.000,100.000,2012-07-05 06:49:00,7/5/2012 6:48,100000.000,201207
9,15,231.000,250.000,2012-07-06 10:02:00,7/5/2012 6:48,100000.000,201207
0,15,20.000,250.000,2012-07-10 17:00:00,7/5/2012 6:48,100000.000,201207
1,15,1539.000,100.000,2012-07-10 21:09:00,7/5/2012 6:48,100000.000,201207
4,15,1418.000,50.000,2012-07-11 08:47:00,7/5/2012 6:48,100000.000,201207
2,15,231.000,850.000,2012-07-17 23:10:00,7/5/2012 6:48,100000.000,201207
5,15,1710.000,200.000,2012-07-21 01:38:00,7/5/2012 6:48,100000.000,201207
7,15,13.000,2030.000,2012-07-23 00:51:00,7/5/2012 6:48,100000.000,201207
8,15,1776.000,20.000,2012-07-23 15:09:00,7/5/2012 6:48,100000.000,201207
6,15,2525.000,50.000,2012-07-24 12:54:00,7/5/2012 6:48,100000.000,201207


In [182]:
def to_limit(row):
    if row['cumsum'] < limit:
        return 1
    else:
        return 0

threshold = .05
dfs=[]
for campaign_id in tqdm_notebook(investments.campaign.unique()):
    x = investments.query("campaign == {0}".format(campaign_id)).sort_values(by='investment_timestamp')
    
    
    x['cumsum'] = x.amount.cumsum()
    try:
        limit = (x['cumsum'][-1])*threshold
        x['needed'] =x.apply(to_limit,1)
        x_c_perc = x[x['needed']==1]
        del x['needed']
        dfs.append(x_c_perc)
    except:
        pass
percent = pd.concat(dfs, axis=0)

In [183]:
# def filter_investments_by_amount(threshold):
#     investments_ = []
#     threshold = threshold
#     for campaign_id in tqdm_notebook(investments.campaign.unique()):
#         var_df = investments.query("campaign == {0}".format(campaign_id)).sort_values(by='investment_timestamp')
# #         series = var_df['amount'].fillna(0)
#         var_df['cumsum'] = var_df.amount.fillna(0).cumsum()
# #         norm_cdf = np.cumsum(series)
#         try:
#             limit = max(series)*threshold
#             # if first contribution is greater than percentage amount
#             if norm_cdf[0] > limit:
#                 pass
#             else:
#                  for i in range(len(norm_cdf)):
#                     if limit < norm_cdf[i]:
#                         investments_.append(var_df[:i-1])  
#                         break
#         except:
#             pass
        
#     return pd.concat(investments_, axis=0)

In [184]:
splits = [.05, .10, .15, .20, .25, .30, .35, .40, .45, .50]
investments_array = []
for split in splits:
    investments_array.append(filter_investments_by_amount(split))

In [185]:
print [len(x) for x in investments_array]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [45]:
# investments_25 = filter_investments_by_amount(0.25)
# investments_50 = filter_investments_by_amount(0.5)
# investments_75 = filter_investments_by_amount(0.75)
# investments_100 = filter_investments_by_amount(1)

In [46]:
# investments_25 = filter_investments(0.25)
# investments_50 = filter_investments(0.5)
# investments_75 = filter_investments(0.75)
# investments_100 = filter_investments(1)

In [47]:
# investments_25.to_csv("../Data/investments_25.csv", index=False)
# investments_50.to_csv("../Data/investments_50.csv", index=False)
# investments_75.to_csv("../Data/investments_75.csv", index=False)
# investments_100.to_csv("../Data/investments_100.csv", index=False)

In [48]:
# investments_25 = pd.read_csv("../Data/investments_25.csv")
# investments_50 = pd.read_csv("../Data/investments_50.csv")
# investments_75 = pd.read_csv("../Data/investments_75.csv")
# investments_100 = pd.read_csv("../Data/investments_100.csv")

In [ ]:
# len(investments_25), len(investments_50), len(investments_75), len(investments_100)

### Investments Aggregation

In [163]:
f = {'campaign': ['max'],
     'investorid':['count'], # Contributor Count
     'approved_timestamp': ['max'], # Investment Approved Timestamp
     'investment_timestamp': ['min', 'max', avgInterEventTime, covInterEventTime], # First Contribution Timestamp, AvgInterEventTime, CovInterEventTime
     'amount': ['count', 'min', 'mean', 'max', 'sum', cov], # Contributions Count, Minimum, Mean, Maximum, CoV
     'amount_requested': ['max'] 
}

In [164]:
def get_investment_array_df(investments_array):
    investments_array_df = []
    for df in tqdm_notebook(investments_array):
        df = df.drop(['time_trunc'], axis=1)
        df.amount = df.amount.astype(float)
        df.amount_requested = df.amount_requested.astype(float)
        df = pd.DataFrame(df.groupby('campaign').agg(f).as_matrix())
        df = df.rename(index=str, columns={0: "AmountRequested",
                                                             1: "ProjectID",
                                                             2: "NumContributions",
                                                             3: "MinContributionAmount",
                                                             4: "MeanContributionAmount",
                                                             5: "MaxContributionAmount",
                                                             6: "AmountRaised",
                                                             7: "CovContributionAmount",
                                                             8: "NumContributors", 
                                                             9: "DatePosted",
                                                             10: "FirstContributionTimestamp",
                                                             11: "LastContributionTimestamp",
                                                             12: "AvgInterEventTime",
                                                             13: "CovInterEventTime"})
        # investments_25["Status"] = investments_25.AmountRaised > investments_25.AmountRequested # This line has to change
        df["TimeToFirstContribution (sec)"] = (pd.to_datetime(df.FirstContributionTimestamp) - pd.to_datetime(df.DatePosted)).dt.seconds
        df["Duration (days)"] = (pd.to_datetime(df.LastContributionTimestamp) - pd.to_datetime(df.DatePosted)).dt.days
        df.ProjectID = df.ProjectID.astype(int)
        investments_array_df.append(df.merge(campaigns, on="ProjectID", how="left"))
    return investments_array_df
        

In [1]:
investments_array_df = get_investment_array_df(investments_array)

In [19]:
for i in range(len(splits)):
    classifier = classifiers.get("RF")
    estimates = []
    X = investments_array_df[i][features_5].fillna(-1).as_matrix()
    X = scaler.fit_transform(X)
    y = investments_array_df[i]['Status'].as_matrix()
    for x in tqdm_notebook(range(0,10)):
        y_pred = run_cv(X,y,classifier)
        estimates.append(auc(y, y_pred))
    print "Mean AUC", np.mean(estimates)
    print "St. Dev", np.std(estimates)
    print "N", len(y)
    print "Investments {0}%".format(splits[i]*100)

Investments 0.05%


Mean AUC 0.591338688085676
St. Dev 0.02409518069276958
N 173
Investments 0.1%


Mean AUC 0.5921052631578948
St. Dev 0.011437197202109929
N 228
Investments 0.15%


Mean AUC 0.596596710153148
St. Dev 0.016748953150508863
N 252
Investments 0.2%


Mean AUC 0.6303964869529902
St. Dev 0.011892418090872986
N 281
Investments 0.25%


...

Mean AUC 0.6065798816568048
St. Dev 0.013370340668565702
N 319
Investments 0.35%


Mean AUC 0.6250689485813593
St. Dev 0.012572669955711826
N 326
Investments 0.4%


Mean AUC 0.625017707890636
St. Dev 0.01352530031450476
N 337
Investments 0.45%


Mean AUC 0.6300862211905156
St. Dev 0.011335579120962893
N 348
Investments 0.5%


Mean AUC 0.6211342780448217
St. Dev 0.00787784357420924
N 359


#### Investments 25%

In [19]:
investments_25 = investments_25.drop(['time_trunc'], axis=1)
investments_25.amount = investments_25.amount.astype(float)
investments_25.amount_requested = investments_25.amount_requested.astype(float)
investments_25 = pd.DataFrame(investments_25.groupby('campaign').agg(f).as_matrix())
investments_25 = investments_25.rename(index=str, columns={0: "AmountRequested",
                                                     1: "ProjectID",
                                                     2: "NumContributions",
                                                     3: "MinContributionAmount",
                                                     4: "MeanContributionAmount",
                                                     5: "MaxContributionAmount",
                                                     6: "AmountRaised",
                                                     7: "CovContributionAmount",
                                                     8: "NumContributors", 
                                                     9: "DatePosted",
                                                     10: "FirstContributionTimestamp",
                                                     11: "LastContributionTimestamp",
                                                     12: "AvgInterEventTime",
                                                     13: "CovInterEventTime"})
# investments_25["Status"] = investments_25.AmountRaised > investments_25.AmountRequested # This line has to change
investments_25["TimeToFirstContribution (sec)"] = (pd.to_datetime(investments_25.FirstContributionTimestamp) - pd.to_datetime(investments_25.DatePosted)).dt.seconds
investments_25["Duration (days)"] = (pd.to_datetime(investments_25.LastContributionTimestamp) - pd.to_datetime(investments_25.DatePosted)).dt.days
investments_25.ProjectID = investments_25.ProjectID.astype(int)
investments_25.head()

,AmountRequested,ProjectID,NumContributions,MinContributionAmount,MeanContributionAmount,MaxContributionAmount,AmountRaised,CovContributionAmount,NumContributors,DatePosted,FirstContributionTimestamp,LastContributionTimestamp,AvgInterEventTime,CovInterEventTime,TimeToFirstContribution (sec),Duration (days)
0,100000.000,15,19,20.000,564.211,2600.000,10720.000,1.327,19,7/5/2012 6:48,2012-07-05 06:49:00,2012-07-21 01:38:00,42163.333,0.656,60,15
1,150000.000,16,7,10.000,501.429,2000.000,3510.000,1.311,7,6/28/2012 10:57,2012-07-04 06:58:00,2012-07-06 07:53:00,14950.000,0.872,72060,7
2,25000.000,38,2,100.000,125.000,150.000,250.000,0.200,2,7/3/2012 10:36,2012-07-04 12:47:00,2012-07-05 10:02:00,76500.000,0.000,7860,1
3,150000.000,49,1,50.000,50.000,50.000,50.000,0.000,1,7/19/2012 11:32,2012-07-24 16:44:00,2012-07-24 16:44:00,NaN,NaN,18720,5
4,30000.000,64,14,20.000,462.143,5000.000,6470.000,2.728,14,7/4/2012 14:35,2012-07-04 17:15:00,2012-07-10 19:10:00,27115.385,0.771,9600,6


#### Investments 50%

In [20]:
investments_50 = investments_50.drop(['time_trunc'], axis=1)
investments_50.amount = investments_50.amount.astype(float)
investments_50.amount_requested = investments_50.amount_requested.astype(float)
investments_50 = pd.DataFrame(investments_50.groupby('campaign').agg(f).as_matrix())
investments_50 = investments_50.rename(index=str, columns={0: "AmountRequested",
                                                     1: "ProjectID",
                                                     2: "NumContributions",
                                                     3: "MinContributionAmount",
                                                     4: "MeanContributionAmount",
                                                     5: "MaxContributionAmount",
                                                     6: "AmountRaised",
                                                     7: "CovContributionAmount",
                                                     8: "NumContributors", 
                                                     9: "DatePosted",
                                                     10: "FirstContributionTimestamp",
                                                     11: "LastContributionTimestamp",
                                                     12: "AvgInterEventTime",
                                                     13: "CovInterEventTime"})
# investments_50["Status"] = investments_50.AmountRaised > investments_50.AmountRequested
investments_50["TimeToFirstContribution (sec)"] = (pd.to_datetime(investments_50.FirstContributionTimestamp) - pd.to_datetime(investments_50.DatePosted)).dt.seconds
investments_50["Duration (days)"] = (pd.to_datetime(investments_50.LastContributionTimestamp) - pd.to_datetime(investments_50.DatePosted)).dt.days
investments_50.ProjectID = investments_50.ProjectID.astype(int)
investments_50.head()

,AmountRequested,ProjectID,NumContributions,MinContributionAmount,MeanContributionAmount,MaxContributionAmount,AmountRaised,CovContributionAmount,NumContributors,DatePosted,FirstContributionTimestamp,LastContributionTimestamp,AvgInterEventTime,CovInterEventTime,TimeToFirstContribution (sec),Duration (days)
0,100000.000,15,19,20.000,564.211,2600.000,10720.000,1.327,19,7/5/2012 6:48,2012-07-05 06:49:00,2012-07-21 01:38:00,42163.333,0.656,60,15
1,150000.000,16,7,10.000,501.429,2000.000,3510.000,1.311,7,6/28/2012 10:57,2012-07-04 06:58:00,2012-07-06 07:53:00,14950.000,0.872,72060,7
2,25000.000,38,4,20.000,75.000,150.000,300.000,0.709,4,7/3/2012 10:36,2012-07-04 12:47:00,2012-07-07 17:19:00,63040.000,0.272,7860,4
3,150000.000,49,2,10.000,30.000,50.000,60.000,0.667,2,7/19/2012 11:32,2012-07-24 16:44:00,2012-07-26 15:41:00,82620.000,0.000,18720,7
4,30000.000,64,25,10.000,348.400,5000.000,8710.000,2.750,25,7/4/2012 14:35,2012-07-04 17:15:00,2012-07-15 19:00:00,32662.500,0.859,9600,11


#### Investments 75%

In [21]:
investments_75 = investments_75.drop(['time_trunc'], axis=1)
investments_75.amount = investments_75.amount.astype(float)
investments_75.amount_requested = investments_75.amount_requested.astype(float)
investments_75 = pd.DataFrame(investments_75.groupby('campaign').agg(f).as_matrix())
investments_75 = investments_75.rename(index=str, columns={0: "AmountRequested",
                                                     1: "ProjectID",
                                                     2: "NumContributions",
                                                     3: "MinContributionAmount",
                                                     4: "MeanContributionAmount",
                                                     5: "MaxContributionAmount",
                                                     6: "AmountRaised",
                                                     7: "CovContributionAmount",
                                                     8: "NumContributors", 
                                                     9: "DatePosted",
                                                     10: "FirstContributionTimestamp",
                                                     11: "LastContributionTimestamp",
                                                     12: "AvgInterEventTime",
                                                     13: "CovInterEventTime"})
# investments_75["Status"] = investments_75.AmountRaised > investments_75.AmountRequested
investments_75["TimeToFirstContribution (sec)"] = (pd.to_datetime(investments_75.FirstContributionTimestamp) - pd.to_datetime(investments_75.DatePosted)).dt.seconds
investments_75["Duration (days)"] = (pd.to_datetime(investments_75.LastContributionTimestamp) - pd.to_datetime(investments_75.DatePosted)).dt.days
investments_75.ProjectID = investments_75.ProjectID.astype(int)
investments_75.head()

,AmountRequested,ProjectID,NumContributions,MinContributionAmount,MeanContributionAmount,MaxContributionAmount,AmountRaised,CovContributionAmount,NumContributors,DatePosted,FirstContributionTimestamp,LastContributionTimestamp,AvgInterEventTime,CovInterEventTime,TimeToFirstContribution (sec),Duration (days)
0,100000.000,15,19,20.000,564.211,2600.000,10720.000,1.327,19,7/5/2012 6:48,2012-07-05 06:49:00,2012-07-21 01:38:00,42163.333,0.656,60,15
1,150000.000,16,7,10.000,501.429,2000.000,3510.000,1.311,7,6/28/2012 10:57,2012-07-04 06:58:00,2012-07-06 07:53:00,14950.000,0.872,72060,7
2,150000.000,35,1,100.000,100.000,100.000,100.000,0.000,1,7/5/2012 11:44,2012-07-05 21:51:00,2012-07-05 21:51:00,NaN,NaN,36420,0
3,25000.000,38,4,20.000,75.000,150.000,300.000,0.709,4,7/3/2012 10:36,2012-07-04 12:47:00,2012-07-07 17:19:00,63040.000,0.272,7860,4
4,150000.000,49,3,10.000,36.667,50.000,110.000,0.514,3,7/19/2012 11:32,2012-07-24 16:44:00,2012-07-27 12:55:00,79530.000,0.039,18720,8


#### Investments 100%

In [22]:
investments_100 = investments_100.drop(['time_trunc'], axis=1)
investments_100.amount = investments_100.amount.astype(float)
investments_100.amount_requested = investments_100.amount_requested.astype(float)
investments_100 = pd.DataFrame(investments_100.groupby('campaign').agg(f).as_matrix())
investments_100 = investments_100.rename(index=str, columns={0: "AmountRequested",
                                                     1: "ProjectID",
                                                     2: "NumContributions",
                                                     3: "MinContributionAmount",
                                                     4: "MeanContributionAmount",
                                                     5: "MaxContributionAmount",
                                                     6: "AmountRaised",
                                                     7: "CovContributionAmount",
                                                     8: "NumContributors", 
                                                     9: "DatePosted",
                                                     10: "FirstContributionTimestamp",
                                                     11: "LastContributionTimestamp",
                                                     12: "AvgInterEventTime",
                                                     13: "CovInterEventTime"})
# investments_100["Status"] = investments_100.AmountRaised > investments_100.AmountRequested
investments_100["TimeToFirstContribution (sec)"] = (pd.to_datetime(investments_100.FirstContributionTimestamp) - pd.to_datetime(investments_100.DatePosted)).dt.seconds
investments_100["Duration (days)"] = (pd.to_datetime(investments_100.LastContributionTimestamp) - pd.to_datetime(investments_100.DatePosted)).dt.days
investments_100.ProjectID = investments_100.ProjectID.astype(int)
investments_100.head()

,AmountRequested,ProjectID,NumContributions,MinContributionAmount,MeanContributionAmount,MaxContributionAmount,AmountRaised,CovContributionAmount,NumContributors,DatePosted,FirstContributionTimestamp,LastContributionTimestamp,AvgInterEventTime,CovInterEventTime,TimeToFirstContribution (sec),Duration (days)
0,100000.000,15,19,20.000,564.211,2600.000,10720.000,1.327,19,7/5/2012 6:48,2012-07-05 06:49:00,2012-07-21 01:38:00,42163.333,0.656,60,15
1,150000.000,16,7,10.000,501.429,2000.000,3510.000,1.311,7,6/28/2012 10:57,2012-07-04 06:58:00,2012-07-06 07:53:00,14950.000,0.872,72060,7
2,150000.000,35,2,100.000,100.000,100.000,200.000,0.000,2,7/5/2012 11:44,2012-07-05 21:51:00,2012-07-05 21:59:00,480.000,0.000,36420,0
3,25000.000,38,7,20.000,142.857,500.000,1000.000,1.061,7,7/3/2012 10:36,2012-07-04 12:47:00,2012-07-11 15:51:00,59440.000,0.417,7860,8
4,150000.000,49,3,10.000,36.667,50.000,110.000,0.514,3,7/19/2012 11:32,2012-07-24 16:44:00,2012-07-27 12:55:00,79530.000,0.039,18720,8


## Merge Data

In [23]:
investments_25 = investments_25.merge(campaigns, on="ProjectID", how="left")
investments_50 = investments_50.merge(campaigns, on="ProjectID", how="left")
investments_75 = investments_75.merge(campaigns, on="ProjectID", how="left")
investments_100 = investments_100.merge(campaigns, on="ProjectID", how="left")
# investments_25.sample(5)

## Model Evaluation

In [50]:
features_5 = ["NumContributors", "CovInterEventTime", "CovContributionAmount", "TimeToFirstContribution (sec)", 'Duration (days)']
# Initialize Standard Scaler
scaler = StandardScaler()

### Investments 25%

In [25]:
X = investments_25[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = investments_25['Status']
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({0.0: 157, 1.0: 145})
Completed QDA in 0.0494782 seconds
Completed CART in 0.0397845 seconds
Completed GNB in 0.0335473 seconds
Completed RF in 19.4169502 seconds
Completed LR in 0.0383458 seconds
Completed ADB in 0.816734 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.613,0.635,0.455,0.530,0.607
1,CART,0.560,0.543,0.524,0.533,0.558
2,GNB,0.656,0.681,0.531,0.597,0.651
3,RF,0.629,0.674,0.441,0.533,0.622
4,LR,0.556,0.824,0.097,0.173,0.539
5,ADB,0.629,0.622,0.579,0.600,0.627


In [38]:
classifier = classifiers.get("RF")
estimates = []
X = investments_25[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = investments_25['Status'].as_matrix()
for x in tqdm_notebook(range(0,100)):
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "N", len(y)

Mean AUC 0.6060742367669668
St. Dev 0.016902528136247562
N 302


### Investments 50%

In [26]:
X = investments_50[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = investments_50['Status']
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({0.0: 193, 1.0: 166})
Completed QDA in 0.0329179 seconds
Completed CART in 0.0486067 seconds
Completed GNB in 0.0390897 seconds
Completed RF in 18.7671425 seconds
Completed LR in 0.0233475 seconds
Completed ADB in 0.9133082 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.638,0.614,0.584,0.599,0.634
1,CART,0.591,0.559,0.542,0.550,0.587
2,GNB,0.638,0.610,0.602,0.606,0.635
3,RF,0.666,0.717,0.458,0.559,0.651
4,LR,0.579,0.800,0.120,0.209,0.547
5,ADB,0.638,0.632,0.518,0.570,0.630


In [39]:
classifier = classifiers.get("RF")
estimates = []
X = investments_50[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = investments_50['Status'].as_matrix()
for x in tqdm_notebook(range(0,100)):
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "N", len(y)

Mean AUC 0.6257166489793371
St. Dev 0.015721433742540588
N 359


### Investments 75%

In [27]:
X = investments_75[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = investments_75['Status']
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({0.0: 228, 1.0: 184})
Completed QDA in 0.0340977 seconds
Completed CART in 0.0314057 seconds
Completed GNB in 0.0343025 seconds
Completed RF in 20.1871647 seconds
Completed LR in 0.0386541 seconds
Completed ADB in 1.0834038 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.667,0.618,0.668,0.642,0.668
1,CART,0.648,0.611,0.582,0.596,0.642
2,GNB,0.658,0.617,0.614,0.616,0.654
3,RF,0.680,0.728,0.451,0.557,0.658
4,LR,0.629,0.830,0.212,0.338,0.588
5,ADB,0.672,0.643,0.598,0.620,0.665


In [40]:
classifier = classifiers.get("RF")
estimates = []
X = investments_75[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = investments_75['Status'].as_matrix()
for x in tqdm_notebook(range(0,100)):
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "N", len(y)

Mean AUC 0.6522459000762776
St. Dev 0.011689403158736764
N 412


### Investments 100%

In [28]:
X = investments_100[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = investments_100['Status']
print Counter(y)
evaluation = evaluation_summary(X,y)
evaluation

Counter({0.0: 253, 1.0: 191})
Completed QDA in 0.1473498 seconds
Completed CART in 0.06752 seconds
Completed GNB in 0.072366 seconds
Completed RF in 25.403917 seconds
Completed LR in 0.0540767 seconds
Completed ADB in 1.3520433 seconds


,Model,Accuracy,Precision,Recall,F1,AUC
0,QDA,0.707,0.660,0.660,0.660,0.701
1,CART,0.637,0.582,0.555,0.568,0.627
2,GNB,0.680,0.640,0.586,0.612,0.669
3,RF,0.721,0.772,0.497,0.605,0.693
4,LR,0.640,0.878,0.188,0.310,0.584
5,ADB,0.685,0.644,0.597,0.620,0.674


In [41]:
classifier = classifiers.get("RF")
estimates = []
X = investments_100[features_5].fillna(-1).as_matrix()
X = scaler.fit_transform(X)
y = investments_100['Status'].as_matrix()
for x in tqdm_notebook(range(0,100)):
    y_pred = run_cv(X,y,classifier)
    estimates.append(auc(y, y_pred))
print "Mean AUC", np.mean(estimates)
print "St. Dev", np.std(estimates)
print "N", len(y)

Mean AUC 0.6855130062289179
St. Dev 0.014159595896781521
N 444
